# SQL Injection Attack

## Introduction

SQL Injection is a technique for attacking applications that utilize a database. The broad principle is to exploit improper user input validation to run arbitrary SQL code on the database. This may allow a malicious user to delete entries or tables from the database, create a record to obtain administrative privileges or simply access data that they are not authorized to view. 

## Outline

The example will use a simple database that stores email addresses for different users along with a username and password. A specific user can obtain their email addresses by providing their username and password to a database client. The database client does not protect against arbitrary SQL code and the user can verify this by running various queries that demonstrate this vulnerability. The user can then secure the client by adding input validation and retry the previous queries which should no longer provide unauthorized access to the database.

## Steps

### Getting Started

1. Open a terminal window and start up the database client at the prompt by typing:
```console
jovyan@host:~$ EmailCloud
```
2. You will then be repeatedly prompted for a ```Username``` and ```password``` combination.
3. First let's try a user that exists in the database; type ```Alice``` for the username and ```pwd``` for the password. You should get Alice's email addresses in response.
4. Next, let's try a user that does not exist in the database. Type ```random``` and ```forgot``` for the username and password respectively. You should see the error message ```Invalid username password pair!```

### Access Unauthorized Content

While everything seems ok so far; you will now see how you can pull up all the data from the database due to vulnerability in the code. 

1. Try ```a' OR '1=1``` for both the username and password. How did this work? (*hint: 1=1 always evaluates to TRUE*)
2. Now let's try to retrieve data without providing a password. For the username, type ```Alice'; --``` and no password. How did this work? (*hint: what is the comment character in SQL?*)

### Modifying the Database

A persistent hacker can also figure out the name of the table containing these records since database tables typically have descriptive names. 

Type ```x' OR 0 < (SELECT COUNT(*) FROM random_name); --``` for the username and use any password, choosing ```random_name``` to be your best guess for the name of the table. (*hint: try ```emailcloud```; how do we know this is the right table name?*)

Once you know the table's name, you can now insert records into the table that may provide you with administrative access to other applications controlled by this table. 

Type ```x'; INSERT INTO emailcloud VALUES ('hacker','pwd','hacker@hacker.org'); --``` for the username and use any password. You should then be able to use ```hacker``` and ```pwd``` as a valid ```username``` and ```password``` when prompted by EmailCloud.

Infact, you can even delete the table using a similar approach:

Type ```x'; DROP TABLE emailcloud; --``` as the username and any password. Now try to retrieve Alice's email address. What happened?

### Recovering the Table

At the terminal window, type:
```console
jovyan@host:~$ RecoverDatabase
```

## Securing the Client

Go back to the file browser window in Jupyter into the ```app``` folder and find the file ```client.py```. Double click on the filename to edit the file contents. Comment these two lines immediately following ```#unsafe``` by placing a ```#``` at the beginning of the line:

```python
command = "SELECT * FROM emailcloud WHERE username = '" + username + "' AND password = '" + password + "'"
sqlite.execute(c, command)
```

Notice that the ```username``` and ```password``` typed by the user are directly substituted in the SQL command. This is the source of the vulnerability that we have been able to exploit. Next, let us protect against SQL injection by sanitizing the inputs. 

Uncomment the two lines following ```#SQL injection protection``` by removing the ```#``` at the beginning of these lines:

```python
# command = "SELECT * FROM emailcloud WHERE username = ? AND password = ?"
# c.execute(command, (username, password))
```

Save the file and then re-run ```EmailCloud```.  
**Note:** You can exit out of ```EmailCloud``` by typing ```ctrl-c``` at the terminal. 

Repeat the previous tests to make sure that the client is no longer vulnerable to SQL injection attacks.

### Review

To learn more about SQL injection, visit the [Wikipedia Page](https://en.wikipedia.org/wiki/SQL_injection)